In [ ]:

import os
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, RepeatVector, TimeDistributed, Dropout
from sklearn.model_selection import train_test_split
import tensorflow as tf
print("Libraries loaded successfully!")


In [ ]:

train_df = pd.read_csv("train_qa.csv")
test_df = pd.read_csv("test_qa.csv")
print("Training data shape:", train_df.shape)
print("Test data shape:", test_df.shape)
train_df.head()


In [ ]:
#Vocabulary and Preprocessing
vocab = set()

train_df["story"] = train_df["story"].astype(str)
train_df["question"] = train_df["question"].astype(str)
train_df["answer"] = train_df["answer"].astype(str)

for _, row in train_df.iterrows():
    vocab.update(row["story"].split())
    vocab.update(row["question"].split())

vocab.update(["yes", "no"])
vocab_len = len(vocab)
print(" Vocabulary size:", vocab_len)

#Tokenization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(list(train_df["story"]) + list(train_df["question"]))

story_maxlen = max([len(s.split()) for s in train_df["story"]])
question_maxlen = max([len(q.split()) for q in train_df["question"]])

def vectorize_data(dataframe, tokenizer, story_maxlen, question_maxlen):
    Xstories, Xquestions, Yanswers = [], [], []
    for _, row in dataframe.iterrows():
        s_seq = tokenizer.texts_to_sequences([row["story"].lower()])[0]
        q_seq = tokenizer.texts_to_sequences([row["question"].lower()])[0]
        ans = 1 if row["answer"].strip().lower() == "yes" else 0
        Xstories.append(s_seq)
        Xquestions.append(q_seq)
        Yanswers.append(ans)
    Xstories = pad_sequences(Xstories, maxlen=story_maxlen)
    Xquestions = pad_sequences(Xquestions, maxlen=question_maxlen)
    Yanswers = np.array(Yanswers)
    return Xstories, Xquestions, Yanswers

Xstory, Xques, Y = vectorize_data(train_df, tokenizer, story_maxlen, question_maxlen)
from sklearn.model_selection import train_test_split
Xtrain_story, Xval_story, Xtrain_ques, Xval_ques, Ytrain, Yval = train_test_split(
    Xstory, Xques, Y, test_size=0.2, random_state=42
)

#Model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embed_size = 128
story_input = Input((story_maxlen,))
question_input = Input((question_maxlen,))

story_encoder = Embedding(vocab_len + 1, embed_size)(story_input)
story_encoder = LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)(story_encoder)
story_encoder = LSTM(64, activation='tanh')(story_encoder)

question_encoder = Embedding(vocab_len + 1, embed_size)(question_input)
question_encoder = LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)(question_encoder)
question_encoder = LSTM(64, activation='tanh')(question_encoder)

merged = concatenate([story_encoder, question_encoder])
merged = Dense(32, activation='relu')(merged)
merged = Dropout(0.3)(merged)
output = Dense(1, activation='sigmoid')(merged)

model = Model([story_input, question_input], output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint("chatbot_model_best.h5", monitor='val_loss', save_best_only=True)
]

history = model.fit(
    [Xtrain_story, Xtrain_ques], Ytrain,
    validation_data=([Xval_story, Xval_ques], Yval),
    epochs=10,
    batch_size=32,
    callbacks=callbacks
)

# Evaluation
Xtest_story, Xtest_ques, Ytest = vectorize_data(test_df, tokenizer, story_maxlen, question_maxlen)
test_loss, test_acc = model.evaluate([Xtest_story, Xtest_ques], Ytest)
print(f"Test Accuracy: {test_acc:.3f}")
print(f"Test Loss: {test_loss:.3f}")


## Step 4: Test or Interact with the Chatbot

In [ ]:

def chatbot_response(story, question):
    model = tf.keras.models.load_model("chatbot_model.h5")
    s_seq = tokenizer.texts_to_sequences([story])
    q_seq = tokenizer.texts_to_sequences([question])
    s_pad = pad_sequences(s_seq, maxlen=story_maxlen)
    q_pad = pad_sequences(q_seq, maxlen=question_maxlen)
    pred = model.predict([s_pad, q_pad])[0][0]
    return "yes" if pred > 0.5 else "no"

story_input = "Sandra got the football there. Mary went to the bedroom."
question_input = "Is Mary in the bedroom?"
print("Chatbot answer:", chatbot_response(story_input, question_input))


In [ ]:

def add_qa(story, question, answer, csv_file="train_qa.csv"):
    df = pd.read_csv(csv_file)
    new_entry = {"story": story, "question": question, "answer": answer}
    df = pd.concat([df, pd.DataFrame([new_entry])], ignore_index=True)
    df.to_csv(csv_file, index=False)
    print(f" Added new QA to {csv_file}")

